## 

In [ ]:
from bs4 import BeautifulSoup
import urllib
import re
import pandas as pd
import requests
from urllib.request import urlopen
from urllib.parse import urlparse
import os

In [ ]:
def get_page(url):
    """
    based on the url, get the page information
    """
    try:
         r = requests.get(url)
        if 'Content-Type' in r.headers and r.headers['Content-Type'] == 'text/html':
            return r.text
        else:
            return None
    except Exception as inst:
        return None

In [ ]:
def get_next_target(page):
    start_link = page.find('<a href=')
    if start_link == -1:
        return None,0
    start_quote = page.find('"',start_link)
    end_quote = page.find('"',start_quote+1)
    url = page[start_quote+1:end_quote]
    return url,end_quote

In [ ]:
def union(p,q):
    for e in q:
        if e not in p:
            p.append(e)

In [ ]:
def get_all_links(page):
    links = []
    while True:
        url,endpos = get_next_target(page)
        if url:
            links.append(url)
            page = page[endpos:]
        else:
            break
    return links

In [ ]:
def crawl_web(seed):
    tocrawl = [seed]
    crawled = []
    while tocrawl:
        page = tocrawl.pop()
        if page not in crawled:
            union(tocrawl,get_all_links(get_page(page)))
            crawled.append(page)
    return crawled

In [ ]:
def crawl_web(seed,max_depth):
    tocrawl = [seed]
    crawled = []
    next_depth = []
    depth = 0
    while tocrawl and depth < max_depth:
        page = tocrawl.pop()
        if page not in crawled:
            union(next_depth,get_all_links(get_page(page))) 
            crawled.append(page)
        if not tocrawl:
            tocrawl,next_depth = next_depth,[]
            depth = depth + 1
    return crawled

### index \ search

In [ ]:
def add_to_index(index,keyword,url):
    for entry in index:
        if entry[0] == keyword:
            entry[1].append(url)
            return
    index.append([keywoed,[url]])

In [ ]:
def look_up(index,keyword):
    for entry in index:
        if entry[0] == keyword:
            return entry[1]
    return []

In [ ]:
def add_page_to_index(index,url,content):
    words = content.split()
    for word in words:
        add_to_index(index,word,url)

In [ ]:
def crawl_web(seed):
    tocrawl = [seed]
    crawled = []
    index = []
    while tocrawl:
        page = tocrawl.pop()
        if page not in crawled:
            content = get_page(page)
            add_page_to_index(index,page,content)
            union(tocrawl,get_all_links(content))
            crawled.append(page)
    return index